In [1]:
import dataset
import importlib
importlib.reload(dataset)
from dataset import DatasetCreation

In [2]:
from torch_geometric.utils import to_undirected,add_self_loops
from torch_geometric.data import Data
import itertools


ModuleNotFoundError: No module named 'torch_geometric'

In [ ]:
#Creating the data, e.g., in this example, we create 1000 graphs with 10 nodes, 1000 with 15, etc.
#We create graphs with varying number of nodes such that the neural network can learn to generalize.
#Ideally we want many graphs and with many different number of nodes, but that can create computational problems if we lack the resources
#or it the GNN we consider have many parameters.
#Also, the bigger the graph, the more time concorde will need for solving (NP-hard problem)

from torch_geometric.utils import to_undirected,add_self_loops
from torch_geometric.data import Data
import itertools




data_list = []

num_datasets = 10

for num_nodes in [10,15,20,25,30]:
  node_coords, distance_matrices,solution_paths, solution_adjacencies, distances = DatasetCreation.generate_tsp_data(num_nodes = num_nodes,  num_datasets = num_datasets, check_correct=True)

  num_pos = num_nodes
  num_neg = num_nodes**2-num_nodes



  weight_pos_class = (num_neg/num_pos)

  edge_index = torch.tensor(list(itertools.product(np.arange(num_nodes),np.arange(num_nodes))), dtype=torch.long).T.contiguous()
  for i in range(len(node_coords)):
      edge_attr = torch.tensor(((distance_matrices[i])).flatten()).float().unsqueeze(1)

      x = torch.tensor(node_coords[i]).float()
      y = torch.tensor(solution_adjacencies[i].flatten()).float().unsqueeze(1)


      data = Data(x=x, edge_index=edge_index, y= y, edge_attr=edge_attr)
      data.edge_weight = torch.tensor(((distance_matrices[i])).flatten()).float().unsqueeze(1)
      data.true_path = torch.Tensor(solution_paths[i])
      data.true_distance = torch.Tensor([distances[i]]).unsqueeze(1)
      data.num_nodes = num_nodes
      data.pos_class_weight = weight_pos_class
      data_list.append(data)